<a href="https://colab.research.google.com/github/naphatsiri/M.Sc-IS-Fault-Detection/blob/main/IS_fault_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.Data preparation

In [1]:
#Import working libraries.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
!wget https://github.com/naphatsiri/M.Sc-IS-Fault-Detection/raw/main/Refrigerant-leak_data/normal%20r.xls
!wget https://github.com/naphatsiri/M.Sc-IS-Fault-Detection/raw/main/Refrigerant-leak_data/rl10.xls

--2025-10-06 07:39:38--  https://github.com/naphatsiri/M.Sc-IS-Fault-Detection/raw/main/Refrigerant-leak_data/normal%20r.xls
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/naphatsiri/M.Sc-IS-Fault-Detection/main/Refrigerant-leak_data/normal%20r.xls [following]
--2025-10-06 07:39:38--  https://raw.githubusercontent.com/naphatsiri/M.Sc-IS-Fault-Detection/main/Refrigerant-leak_data/normal%20r.xls
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3102208 (3.0M) [application/octet-stream]
Saving to: ‘normal r.xls.1’

normal r.xls.1      100%[===================>]   2.96M  --.-KB/s    in 0.01s   

2025

In [27]:
#Use rl10% and rl_normal dataset

df_normal = pd.read_excel("/content/normal r.xls.1", sheet_name="Reduced Data Set", engine='xlrd')
df_fault = pd.read_excel("/content/rl10.xls.1", sheet_name="Reduced Data Set", engine='xlrd')

In [14]:
df_normal.info()
df_fault.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 66 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Time (minutes)       433 non-null    float64
 1   TWE_set              433 non-null    int64  
 2   TEI                  433 non-null    float64
 3   TWEI                 433 non-null    float64
 4   TEO                  433 non-null    float64
 5   TWEO                 433 non-null    float64
 6   TCI                  433 non-null    float64
 7   TWCI                 433 non-null    float64
 8   TCO                  433 non-null    float64
 9   TWCO                 433 non-null    float64
 10  TSI                  433 non-null    float64
 11  TSO                  433 non-null    float64
 12  TBI                  433 non-null    float64
 13  TBO                  433 non-null    float64
 14  Cond Tons            433 non-null    float64
 15  Cooling Tons         433 non-null    flo

In [28]:
df_normal["label"] = 0
df_fault["label"] = 1

df = pd.concat([df_normal, df_fault], ignore_index=True)

In [29]:
#Clean and filter the data : removen oise, non-operational periods, and impossible values.

# 1.Keep only periods when chiller was running
df = df[df["Unit Status"] == 1]

# 2.Drop rows with zero cooling load or zero power (non-operational or startup)
df = df[(df["Cooling Tons"] > 0) & (df["kW"] > 0)]

# 3.Capture extreme values for key sensors (99.9th percentile) to prevent model skew
for col in ["PRE", "PRC", "kW", "Cooling Tons", "COP", "Amps"]:
    lower, upper = df[col].quantile([0.001, 0.999])
    df[col] = df[col].clip(lower, upper)

'''4. Drop irrelevant or constant columns (optional)
drop_cols = ["Active Fault", "Heat Balance%", "Tolerance%"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])
'''

# 5. Sort by time
df = df.sort_values(by="Time (minutes)").reset_index(drop=True)

print("Cleaned dataset shape:", df.shape)


Cleaned dataset shape: (2, 67)


In [30]:
df.head()

,Time (minutes),TWE_set,TEI,TWEI,TEO,TWEO,TCI,TWCI,TCO,TWCO,...,VE,VW,TWI,TWO,THI,THO,FWW,FWH,FWB,label
0,0.0597,50,68.87,69.0,69.43,69.4,69.89,69.8,70.55,70.6,...,56,0.0,67.40,71.63,69.87,71.00,4.618000e-46,1.096000e-45,3.664000e-45,1
1,0.0612,50,65.59,65.6,66.40,66.4,67.44,67.3,68.50,68.4,...,56,0.0,66.21,71.01,67.54,70.72,2.202000e-47,5.491000e-46,3.853000e-45,0


In [31]:
# Select the relevant columns for ML

# Core measured features for refrigerant leak detection
selected_features = [
    "TEI", "TEO", "TWI", "TWO",           # Thermal temperatures
    "TCI", "TCO", "TWCI", "TWCO",         # Condenser temps
    "PRE", "PRC", "P_lift",               # Pressure features
    "kW", "COP", "Cooling Tons",          # Performance indicators
    "Amps", "RLA%",                       # Electrical features
    "Tsh_suc", "Tsh_dis",                 # Superheat
    "kW/Ton"                              # Efficiency
]

df = df[["Time (minutes)"] + selected_features + ["label"]]
print("Final feature set:", df.shape)


Final feature set: (2, 21)


In [32]:
df.head()

,Time (minutes),TEI,TEO,TWI,TWO,TCI,TCO,TWCI,TWCO,PRE,...,P_lift,kW,COP,Cooling Tons,Amps,RLA%,Tsh_suc,Tsh_dis,kW/Ton,label
0,0.0597,68.87,69.43,67.40,71.63,69.89,70.55,69.8,70.6,70.3957,...,0.0,1.682000e-45,2.535115e-46,8.129304e-47,0,0,15.3,4.4,1.682000e-45,1
1,0.0612,65.59,66.40,66.21,71.01,67.44,68.50,67.3,68.4,66.1043,...,0.0,1.682000e-45,3.647885e-46,4.482964e-48,0,0,16.1,6.4,1.682000e-45,0


In [21]:
#To confirm that there is no missing value.

print(df.isna().sum())

Time (minutes)    0
TEI               0
TEO               0
TWI               0
TWO               0
TCI               0
TCO               0
TWCI              0
TWCO              0
PRE               0
PRC               0
P_lift            0
kW                0
COP               0
Cooling Tons      0
Amps              0
RLA%              0
Tsh_suc           0
Tsh_dis           0
kW/Ton            0
label             0
dtype: int64


## 2.Features engineering operation

In [33]:
# Derived temperature features

# Evaporator and condenser approach temperatures
df["Evap_Approach"] = df["TEO"] - df["TEI"]        # Evaporator approach (ΔT across evaporator)
df["Cond_Approach"] = df["TCO"] - df["TCI"]        # Condenser approach (ΔT across condenser)

# Temperature lift: overall temperature difference between condenser and evaporator
df["Temp_Lift"] = df["TCO"] - df["TEO"]

# Pressure-based features
df["Pressure_Ratio"] = df["PRC"] / df["PRE"].replace(0, np.nan)  # Prevent divide-by-zero

# Efficiency-based features
df["kW_per_Ton"] = df["kW"] / df["Cooling Tons"].replace(0, np.nan)

# Superheat difference
df["Delta_Superheat"] = df["Tsh_suc"] - df["Tsh_dis"]


In [34]:
df.head()

,Time (minutes),TEI,TEO,TWI,TWO,TCI,TCO,TWCI,TWCO,PRE,...,Tsh_suc,Tsh_dis,kW/Ton,label,Evap_Approach,Cond_Approach,Temp_Lift,Pressure_Ratio,kW_per_Ton,Delta_Superheat
0,0.0597,68.87,69.43,67.40,71.63,69.89,70.55,69.8,70.6,70.3957,...,15.3,4.4,1.682000e-45,1,0.56,0.66,1.12,0.994322,20.690579,10.9
1,0.0612,65.59,66.40,66.21,71.01,67.44,68.50,67.3,68.4,66.1043,...,16.1,6.4,1.682000e-45,0,0.81,1.06,2.10,0.998483,375.198195,9.7


In [24]:
# Create Rolling (Time-Series) Features

# Define columns to create rolling stats for
rolling_cols = ["COP", "kW", "Cooling Tons", "PRE", "PRC"]

# Create rolling mean and standard deviation (window ~6 samples ≈ 1 min)
for col in rolling_cols:
    df[f"{col}_mean"] = df[col].rolling(window=6, min_periods=1).mean()
    df[f"{col}_std"] = df[col].rolling(window=6, min_periods=1).std()


In [25]:
# Final features of operational data.

feature_cols = [
    # Raw sensor features
    "TEI", "TEO", "TWI", "TWO", "TCI", "TCO",
    "PRE", "PRC", "P_lift", "kW", "COP", "Cooling Tons",
    "Amps", "RLA%", "Tsh_suc", "Tsh_dis", "kW/Ton",

    # Derived features
    "Evap_Approach", "Cond_Approach", "Temp_Lift",
    "Pressure_Ratio", "kW_per_Ton", "Delta_Superheat",

    # Rolling features
    "COP_mean", "COP_std", "kW_mean", "kW_std",
    "PRE_mean", "PRE_std", "PRC_mean", "PRC_std"
]

# Drop any NaN rows created by rolling calculations
df = df.dropna(subset=feature_cols)

print("Final feature matrix shape:", df[feature_cols].shape)


Final feature matrix shape: (1, 31)


In [26]:
df.head()

,Time (minutes),TEI,TEO,TWI,TWO,TCI,TCO,TWCI,TWCO,PRE,...,COP_mean,COP_std,kW_mean,kW_std,Cooling Tons_mean,Cooling Tons_std,PRE_mean,PRE_std,PRC_mean,PRC_std
1,0.0612,65.59,66.4,66.21,71.01,67.44,68.5,67.3,68.4,66.1043,...,3.091500e-46,7.868472e-47,1.682000e-45,0.0,4.288800e-47,5.431292e-47,68.25,3.034478,68.0,2.82277


## 3.ML model operation

## 4.Evaluation mectrics

## 5.Data visualization